# Deploy and Monitor your Model

In this notebook we will deploy the fashion MNIST model to an Azure Container Instance (ACI), call it using a HTTP request and montior the traffic to the API using Azure Application Insights

The two relevant MS Learn Module for this notebook are:
* [Deploy real-time machine learning services with Azure Machine Learning](https://docs.microsoft.com/en-gb/learn/modules/register-and-deploy-model-with-amls/)
* [Monitor models with Azure Machine Learning](https://docs.microsoft.com/en-gb/learn/modules/monitor-models-with-azure-machine-learning/)

### Connect to your Azure ML Workspace

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

### List the models you have registered

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

### Get the latest Fashion MNIST Model

In [ ]:
model = ws.models['fashion_mnist_model']
print(model.name, 'version', model.version)

### Create a local folder for deployment file and build a score.py file

In [ ]:
import os

folder_name = 'fashion_mnist_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_fashion_mnist.py")

### Create a environment description for the container this model will run in

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')
myenv.add_conda_package('tensorflow')
myenv.add_conda_package('keras')
myenv.add_conda_package('matplotlib')

# Save the environment config as a .yml file
env_file = os.path.join(experiment_folder,"fashion_mnist_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

### Deploy the model to an Azure Container Instance

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "fashion-mnist-service"

print(ws)
print(service_name)
print(model)

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

In [ ]:
print(service.get_logs())

### Call the web service with a HTTP request

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

In [ ]:
## Prepare the test data to be fed into the deployed model

import tensorflow as tf
import os
import time
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
print("tensorflow Version is: " + str(tf.__version__))

import numpy as np
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras import backend as K
print(os.environ['KERAS_BACKEND'])

#Fashion MNIST Dataset CNN model development: https://github.com/zalandoresearch/fashion-mnist
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import utils, losses, optimizers
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#no. of classes
num_classes = 10
# Define the text labels
fashion_mnist_labels = ["Top",          # index 0
                        "Trouser",      # index 1
                        "Jumper",       # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Trainer",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9

x_test = x_test.astype('float32')
x_test /= 255
y_test = utils.to_categorical(y_test,  num_classes)

data = x_test[5].tolist()

In [ ]:
import requests
import json

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": data})


# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

print(predicted_classes)

### Enable Application Insights to Monitor your model

In [ ]:
# Enable AppInsights
service.update(enable_app_insights=True)
print('AppInsights enabled!')